In [120]:
%serialconnect to --port=/dev/ttyUSB0 --baud=115200

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [121]:
# Complete project details at https://RandomNerdTutorials.com

try:
  import usocket as socket
except:
  import socket

from machine import Pin
import network

import esp
esp.osdebug(None)

import utime

import ujson

import gc
gc.collect()



In [122]:
sta_if = network.WLAN(network.STA_IF)
ap_if = network.WLAN(network.AP_IF)
print('Station Up: ', sta_if.active())
print('Acess Point Up: ', ap_if.active())

Station Up:  False
Acess Point Up:  False


In [123]:
def connectX(typeX):
    typeX.active(True)
    print('Station Up: ', sta_if.active())
    print('Acess Point Up: ', ap_if.active())
    
def scanX():
    print('Status: %s\n' % sta_if.status())
    station_default = False
    stations_list = sta_if.scan()
    for i, n in enumerate(stations_list, start=0):
        print('%s) %s' %(i, n))
        if (n[0]) == b'SM-A1kerr':
            station_default = True

    if station_default == False:
        print('*** PLEASE ENABLE DEFAULT STATION! ***')
    return stations_list
    

In [124]:
connectX(sta_if)
stations_list = []


if sta_if.active():
#     print('Station Up: ', sta_if.active())
    stations_list = scanX()
#     print(stations_list)
#     print(sta_if.status())
elif ap_if.active():
    print('Acess Point Up: ', ap_if.active())

Station Up:  True
Acess Point Up:  False
Status: 1000

0) (b'SM-A1kerr', b'\x8e\xf5\xa3\xa2\x7fn', 1, -42, 3, False)
1) (b'Tim Hortons WiFi', b'\xe0\xcb\xbc3\xd0t', 11, -58, 0, False)
2) (b'TDLTablet', b'\x08[\x0e\xe1\xe8*', 11, -68, 4, False)
3) (b'SmartRG-36a5', b'\xe8,m,6\xa8', 1, -88, 4, False)
4) (b'360\xe8\xa1\x8c\xe8\xbd\xa6\xe8\xae\xb0\xe5\xbd\x95\xe4\xbb\xaa-9586', b'\xd0\xfa\x1d\xdd\x95\x86', 1, -89, 0, False)
5) (b'DIRECT-B3-HP OfficeJet Pro 8710', b'@\xb04{\x19\xb4', 11, -91, 3, False)


In [125]:
print('Connecting to: ', stations_list[0][0])
while not(sta_if.isconnected()):
    print('Trying to connet to server!')
    sta_if.connect(stations_list[0][0], '9054830760')
    utime.sleep(2)

print('Status: %s\n' % sta_if.status())
print('Is connected: ', sta_if.isconnected())
print('Connected to: ', sta_if.config('essid'))
print('ifconfig: ', sta_if.ifconfig())


Connecting to:  b'SM-A1kerr'
Trying to connet to server!
Status: 1010

Is connected:  True
Connected to:  SM-A1kerr
ifconfig:  ('192.168.43.34', '255.255.255.0', '192.168.43.1', '192.168.43.1')


<br>
<h1>Websocket Return Header Format</h1>
<p>
HTTP/1.1 101 Switching Protocols<br>
Upgrade: websocket<br>
Connection: Upgrade<br>
Sec-WebSocket-Accept: s3pPLMBiTxaQ9kYGzzhZRbK+xOo=<br>
(remember each header line ends with \r\n and put an extra \r\n after the last one to indicate the end of the header)
</p>
<p>
-hashed key- The server takes the value of Sec-WebSocket-Key sent in handshake request and then appends Globally Unique Identifier (GUID, [RFC4122]) “258EAFA5-E914-47DA-95CA-C5AB0DC85B11” takes SHA-1 of the new value, and is then base64 encoded. This is unlikely to be used by network endpoints that do not understand the WebSocket Protocol.
</p>

In [126]:
# Complete project details at https://RandomNerdTutorials.com

import socket
addr_socket = socket.getaddrinfo('192.168.43.34', 80)[0][4]
# addr = socket.getaddrinfo('192.168.43.34', 80)
led = Pin(2, Pin.OUT)

def web_page():
  if led.value() == 1:
    gpio_state="ON"
  else:
    gpio_state="OFF"
  
  html = """<html><head> <title>ESP Web Server</title> <meta name="viewport" content="width=device-width, initial-scale=1">
  <link rel="icon" href="data:,"> <style>html{font-family: Helvetica; display:inline-block; margin: 0px auto; text-align: center;}
  h1{color: #0F3376; padding: 2vh;}p{font-size: 1.5rem;}.button{display: inline-block; background-color: #e7bd3b; border: none; 
  border-radius: 4px; color: white; padding: 16px 40px; text-decoration: none; font-size: 30px; margin: 2px; cursor: pointer;}
  .button2{background-color: #4286f4;}</style></head><body> <h1>ESP Web Server</h1> 
  <p>GPIO state: <strong>""" + gpio_state + """</strong></p><p><a href="/?led=on"><button class="button">ON</button></a></p>
  <p><a href="/?led=off"><button class="button button2">OFF</button></a></p></body></html>"""
  return html

html2 = """<!DOCTYPE html>
<html>
    <head> <title>ESP8266 Pins</title> </head>
    <body> <h1>ESP8266 Pins</h1>
        <table border="1"> <tr><th>Pin</th><th>Value</th></tr> %s </table>
    </body>
</html>
"""

s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
# s.bind(('', 80))
# s.bind(addr)
s.bind(('192.168.43.34', 80))
print('Bind: ', addr_socket)
# print('Bind: %s' % addr[0])
# print('Bind: %s' % addr[0][-1])
s.listen(5)
socketX_upgrade = bytes('HTTP/1.1 101 Switching Protocols\r\nUpgrade: websocket\r\nConnection: Upgrade\r\nSec-WebSocket-Accept: s3pPLMBiTxaQ9kYGzzhZRbK+xOo=\r\n\r\n', 'utf8')
print('Upgrage Header: ', socketX_upgrade)

while True:
    # #      clientX = s.accept()
    conn, addr = s.accept()
    print('Got a connection!: ')
    #      print('Address: %s' % addr[0])
    #      print('Address: %s' % addr[1])
    # #      print(clientX[1])
    # #      print(' %s' % str(addr))
    print('Got a connection from %s' % str(addr))
    request = conn.recv(1024)
#     request = request.decode()
    print('Content = %s' % request)
    print('Decode Content = %s' % request.decode().split(':')[0])
    conn.send(socketX_upgrade)
    conn.send("I am SERVER\n")
    conn.close()
    #   led_on = request.find('/?led=on')
    #   led_off = request.find('/?led=off')
    #   if led_on == 6:
    #     print('LED ON')
    #     led.value(1)
    #   if led_off == 6:
    #     print('LED OFF')
    #     led.value(0)
    #   response = web_page()
#      conn.send('HTTP/1.1 200 OK\n')
#      conn.send('Content-Type: text/html\n')
#      conn.send('Connection: close\n\n')
    #   conn.sendall(response)
print('Server closed: ')
        

Bind:  ('192.168.43.34', 80)
Upgrage Header:  b'HTTP/1.1 101 Switching Protocols\r\nUpgrade: websocket\r\nConnection: Upgrade\r\nSec-WebSocket-Accept: s3pPLMBiTxaQ9kYGzzhZRbK+xOo=\r\n\r\n'
..Got a connection!: 
Got a connection from ('192.168.43.84', 47830)
Content = b'GET / HTTP/1.1\r\nHost: 192.168.43.34\r\nConnection: Upgrade\r\nPragma: no-cache\r\nCache-Control: no-cache\r\nUser-Agent: Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.87 Safari/537.36\r\nUpgrade: websocket\r\nOrigin: chrome-extension://dpnkggieociffokefcfiocfkppakljgm\r\nSec-WebSocket-Version: 13\r\nAccept-Encoding: gzip, deflate\r\nAccept-Language: en-GB,en-US;q=0.9,en;q=0.8,da;q=0.7\r\nSec-WebSocket-Key: 4Z+Bm7BJ83vLhD9JzjWEPg==\r\nSec-WebSocket-Extensions: permessage-deflate; client_max_window_bits\r\n\r\n'


Traceback (most recent call last):
  File "<stdin>", line 55, in <module>
IndexError: list index out of range


<br>
<p>
Format of getaddrinfo:  [(2, 1, 0, '192.168.43.34', ('192.168.43.34', 80))]
</p>